## Importar librerias

In [0]:
# https://github.com/dssg/hitchhikers-guide/tree/master/sources/curriculum/0_before_you_start/pipelines-and-project-workflow
# https://medium.com/swlh/how-to-structure-a-python-based-data-science-project-a-short-tutorial-for-beginners-7e00bff14f56
# https://drivendata.github.io/cookiecutter-data-science/#data-is-immutable

In [0]:
!pip install -U keras
!pip install -q fake_useragent
!pip install -q aiml

In [0]:
import tensorflow
import keras
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import numpy as np
import random
import json
import pickle
from nltk.stem import SnowballStemmer
import pandas as pd
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import urllib
import os
import ssl
import aiml

In [0]:
os.chdir("/content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/notebooks/")

In [5]:
import aiml
k=aiml.Kernel()
k.learn(os.getcwd()+"/../std-startup.xml")
k.respond("LOAD AIML B")

Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/notebooks/../std-startup.xml...done (0.00 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/65percent.aiml...done (0.03 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/chismes.aiml...done (0.00 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/conocimiento.aiml...done (0.03 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/default.aiml...done (0.00 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/deportes.aiml...done (0.01 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/geografia.aiml...done (0.02 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/inventos.aiml...done (0.00 seconds)
Loading /content/drive/My Drive/Colab Notebooks/Chatbot_BCRP/aiml-set/nombres.aiml...done (0.01 seconds)
Loading /content/drive/My Drive/Colab

''

## Cargar la metadata de los PBI por sectores (mensual)

In [6]:
df = pd.read_excel('../data/PBI_Sectores_ParaPresentarX2.xlsx')
print(df.shape)
df.head()

(120, 13)


,Código de serie,Categoría de serie,Grupo de serie,Nombre de serie,Fuente,Frecuencia,Fecha de creación,Grupo de publicación,Área que publica,Fecha de actualización,Fecha de inicio,Fecha de fin,Columna1
0,PN01713AM,PBI por sectores,Producto bruto interno y demanda interna (vari...,Agropecuario,INEI,Mensual,2016-10-30,Producción,Departamento de Indicadores de la Actividad Ec...,2018-09-20,2004-04-01,2017-09-01,NaN
1,PN01714AM,PBI por sectores,Producto bruto interno y demanda interna (vari...,Agropecuario - Agrícola,INEI,Mensual,2016-10-30,Producción,Departamento de Indicadores de la Actividad Ec...,2018-09-20,2004-04-01,2017-09-01,NaN
2,PN01715AM,PBI por sectores,Producto bruto interno y demanda interna (vari...,Agropecuario - Pecuario,INEI,Mensual,2016-10-30,Producción,Departamento de Indicadores de la Actividad Ec...,2018-09-20,2004-04-01,2017-09-01,NaN
3,PN01716AM,PBI por sectores,Producto bruto interno y demanda interna (vari...,Pesca,INEI,Mensual,2016-10-30,Producción,Departamento de Indicadores de la Actividad Ec...,2018-09-20,2004-04-01,2017-09-01,NaN
4,PN01717AM,PBI por sectores,Producto bruto interno y demanda interna (vari...,Minería e Hidrocarburos,INEI,Mensual,2016-10-30,Producción,Departamento de Indicadores de la Actividad Ec...,2018-09-20,2004-04-01,2017-09-01,NaN


## Crear la Base de Conocimiento del Chatbot

In [0]:
try:
    with open("../data/data_pbi.json","r") as f:
        dict_pbi_=json.load(f)
except:
    with open("../data/data_pbi.json","w") as f:
        dict_pbi_={}
        contents=[]
        for codigo,metrica,nombre,frecuencia in zip(df["Código de serie"],df["Grupo de serie"],df["Nombre de serie"],df["Frecuencia"]):
            metrica=metrica.split("(")[-1][0:-1].strip()
            nombre=nombre.split("-")[-1].strip()
            if "Global" in nombre:
                nombre=nombre.replace("Global"," ")
            if metrica=="miles de unidades recuperables":
                metrica=nombre.split("(")[-1][0:-1].strip()
                nombre=nombre.split("(")[0].strip()
            item={}
            item["tag"]=codigo
            item["patterns"]=["¿Cuánto es el pbi {}?".format(nombre),"¿Cuánto es la producción {}?".format(nombre), "Dime el pbi {}".format(nombre),
                              " ¿A cuánto equivale el pbi {}?".format(nombre),"¿Quiero saber el pbi del sector {}?".format(nombre),
                              "¿Cuánto es el pbi del sector {}?".format(nombre),"¿Cuanto es el Producto Bruto Interno {}?".format(nombre)]
            if metrica=="miles de toneladas" or metrica=="miles de unidades recuperables":
                item["responses"]=["la cantidad de {} producidas por el sector {} es de".format(metrica,nombre),
                                   "El sector {} tiene una produccion {} de ".format(nombre,metrica),
                                  "la produccion en {} del sector {} equivale a".format(metrica,nombre)]
            else: 
                item["responses"]=["La {} del sector {} equivale a".format(metrica,nombre),
                                   "El sector {} tiene {} de".format(nombre,metrica),
                                   "El PBI {} tiene {} de".format(nombre,metrica)]
            contents.append(item)
        item={}
        item["tag"]="saludo1"
        item["patterns"]=["Hola","Saludos","Buenos dias","Buenas tardes","Buenas noches"] ##Buenos dias, 0.69
        item["responses"]=["Hola","Saludos"]
        contents.append(item)

        item={}
        item["tag"]="saludo2"
        item["patterns"]=["Hola, Cómo estas?","Hola, Cómo te va?","Cómo estas?","Cómo te ha ido","Cómo has estado?"]
        item["responses"]=["Bien","Genial","Muy bien gracias","Normal, pero me podría ir mejor","Supongo que bien","Supongo que bien, y cómo estas?"]
        contents.append(item)

        item={}
        item["tag"]="saludos3"
        item["patterns"]=["Un placer conocerte","Hola, encantado de conocerte","Es un placer conocerte","bien"]#encantado de conocerte
        item["responses"]=["Gracias","Gracias. Igualmente","Muy bien, gracias"]    
        contents.append(item)

        dict_pbi_["intents"]=contents
        json.dump(dict_pbi_, f)

In [8]:
dict_pbi_

{'intents': [{'patterns': ['¿Cuánto es el pbi Agropecuario?',
    '¿Cuánto es la producción Agropecuario?',
    'Dime el pbi Agropecuario',
    ' ¿A cuánto equivale el pbi Agropecuario?',
    '¿Quiero saber el pbi del sector Agropecuario?',
    '¿Cuánto es el pbi del sector Agropecuario?',
    '¿Cuanto es el Producto Bruto Interno Agropecuario?'],
   'responses': ['La variaciones porcentuales anualizadas del sector Agropecuario equivale a',
    'El sector Agropecuario tiene variaciones porcentuales anualizadas de',
    'El PBI Agropecuario tiene variaciones porcentuales anualizadas de'],
   'tag': 'PN01713AM'},
  {'patterns': ['¿Cuánto es el pbi Agrícola?',
    '¿Cuánto es la producción Agrícola?',
    'Dime el pbi Agrícola',
    ' ¿A cuánto equivale el pbi Agrícola?',
    '¿Quiero saber el pbi del sector Agrícola?',
    '¿Cuánto es el pbi del sector Agrícola?',
    '¿Cuanto es el Producto Bruto Interno Agrícola?'],
   'responses': ['La variaciones porcentuales anualizadas del sector A

## Preprocesamiento de la Base de Conocimiento

In [0]:
nltk.download('stopwords')
nltk.download('punkt')
try :
    stemmer = SnowballStemmer('spanish') 
    with open("../data/data_pbi.pickle","rb") as f:
        words, labels,training, output = pickle.load(f)
except:
    stemmer = SnowballStemmer('spanish') 
    stop_w=set(stopwords.words("spanish"))
    special_character='!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    stop_w=stop_w.union("(",")",",","-",".","?")

    words =[]
    labels=[]
    docs_x=[]
    docs_y=[]
    for intent in dict_pbi_["intents"]:
        for pattern in intent["patterns"]:
            wrds=word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
        if intent["tag"] not in labels:
            labels.append(intent["tag"])
    words=[stemmer.stem(w.lower()) for w in words if w not in stop_w.difference("no","otros")]
    words =sorted(list(set(words)))
    labels=sorted(labels)
    training=[]
    output=[]
    out_empty=[0 for _ in range(len(labels))]
    for x, doc in enumerate(docs_x):
        bag=[]
        wrds=[stemmer.stem(w.lower()) for w in doc if w not in stop_w.difference("no","otros")]
        for w in words:
            if w in wrds:
                val=1
                bag.append(val)
            else:
                bag.append(0)
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        training.append(bag)
        output.append(output_row)
    training=np.array(training)
    output=np.array(output)
    with open("../data/data_pbi.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)

In [10]:
training.shape

(854, 179)

In [11]:
  output.shape

(854, 123)

## Configuración de las redes neuronales

In [0]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD
from keras.models import load_model
model = Sequential()
model.add(Dense(1024,input_dim=len(training[0]),activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(len(output[0]),activation="softmax"))

In [0]:
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

## Entrenamiento del Modelo

In [14]:
try:
    model=load_model("../models/model.h5")
except:
    model.fit(training,output,epochs=1000,batch_size=32)
    model.save("../models/model.h5")

Epoch 1/1000
854/854 [==============================] - 1s 808us/step - loss: 4.5394 - accuracy: 0.2248
Epoch 2/1000
854/854 [==============================] - 0s 462us/step - loss: 2.9573 - accuracy: 0.7237
Epoch 3/1000
854/854 [==============================] - 0s 454us/step - loss: 1.3090 - accuracy: 0.9309
Epoch 4/1000
854/854 [==============================] - 0s 479us/step - loss: 0.4554 - accuracy: 0.9672
Epoch 5/1000
854/854 [==============================] - 0s 450us/step - loss: 0.1751 - accuracy: 0.9789
Epoch 6/1000
854/854 [==============================] - 0s 450us/step - loss: 0.0865 - accuracy: 0.9918
Epoch 7/1000
854/854 [==============================] - 0s 463us/step - loss: 0.0700 - accuracy: 0.9895
Epoch 8/1000
854/854 [==============================] - 0s 454us/step - loss: 0.0347 - accuracy: 0.9895
Epoch 9/1000
854/854 [==============================] - 0s 450us/step - loss: 0.0353 - accuracy: 0.9883
Epoch 10/1000
854/854 [==============================] - 0s 442u

## Busqueda de la Respuesta

In [0]:
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words)) ]
    s_words=nltk.word_tokenize(s)
    s_words=[stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=1
    return np.array(bag)

## Consumir el servicio web de BCRPData

In [0]:
def buscarDato(codigo,annio,mes):
    dict_mes={"enero":1,"febrero":2,"marzo":3,"abril":4,"mayo":5,"junio":6,"julio":7,"agosto":8,"setiembre":9,"octubre":10,"noviembre":11,"diciembre":12}
    ua=UserAgent()
    gcontext = ssl.SSLContext()
    url="https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{}/json/{}-{}".format(codigo,annio,dict_mes[mes])
    user_agent=ua.chrome
    request = urllib.request.Request(url, headers = {"User-Agent":user_agent})
    page=urllib.request.urlopen(request,context=gcontext)
    soup = BeautifulSoup(page, "html.parser")
    json_text=soup.get_text()
    json_data = json.loads(json_text)
    data=round(float("".join(json_data["periods"][0]["values"])),2)
    return data

In [0]:
meses=["enero","febrero","marzo","abril","mayo","junio","julio","agosto","setiembre","octubre","noviembre","diciembre"]
annios=['1995','1996','1997','1998','1999','2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016', '2017','2018','2019','2020']

## Interaccion

In [0]:
def chat():
    print("Hola soy el chatbot económico del BCRP, puedes consultar sobre cualquier")
    print("información respecto a los sectores del pbi")
    print("========================================================================")
    annio=""
    mes=""
    response=""
    tag=""
    while True:
        inp=input("You: ").lower()
        aux=inp
        if inp == "salir":
            break
        inp=inp.replace("".join([i for i in meses if i in inp]),"")
        inp=inp.replace("".join([i for i in annios if i in inp]),"")
        
        results=model.predict(bag_of_words(inp,words).reshape(1,len(training[0])))
        if np.max(results)>=0.95:
            try:
                results_index = np.argmax(results)
                tag = labels[results_index]
                if "saludo" not in tag:
                    if tag[-1]=="M":
                        mes="".join([i for i in meses if i in aux])
                        annio="".join([i for i in annios if i in aux])                      
                        if mes=="" and annio=="":
                            print("                              Gti_Bot: Por favor puede introducir mes y año")
                            mes_annio=input("You: ").lower()
                            mes="".join([ i for i in meses if i in mes_annio])
                            annio="".join([ i for i in annios if i in mes_annio])
                        if mes=="" and annio!="":
                            print("                              Gti_Bot: Por favor introducir mes")
                            mes=input("You: ").lower()
                            mes="".join([ i for i in meses if i in mes])
                        if annio=="" and mes!="":
                            print("                              Gti_Bot: Por favor introducir año")
                            annio=input("You: ").lower()
                            annio="".join([ i for i in annios if i in annio])
                        for  tg in dict_pbi_["intents"]:
                            if tg["tag"]== tag:
                                  responses = tg["responses"]
                        val=buscarDato(tag,annio,mes)
                    print("                              Gti_Bot: {} {}   ".format(random.choice(responses), val,np.max(results)))
                else:
                    for  tg in dict_pbi_["intents"]:
                        if tg["tag"]== tag:
                                responses = tg["responses"]
                    print("                               Gti_Bot: {}  ".format(random.choice(responses),np.max(results)))
            except:
                print("                               Gti_Bot: disculpe, pero esa información todavía no está disponible en el BCRP Data codigo :{} confidence : {} ".format(tag , np.max(results)))
        elif tag!="":
            mes2="".join([ i for i in meses if i in aux])
            annio2="".join([ i for i in annios if i in aux])
            if mes2!="" or annio2 !="":     
                for  tg in dict_pbi_["intents"]:
                    if tg["tag"]== tag:
                            responses = tg["responses"]
                if mes2!="":
                    mes=mes2
                if annio2!="":
                    annio=annio2
                val=buscarDato(tag,annio,mes)
                print("                              Gti_Bot: {} {} ".format(random.choice(responses), val,np.max(results)))         
            else :
                cadena=k.respond(inp)
                print("                               Gti_Bot: {} ".format(cadena)) 

In [41]:
chat()

Hola soy el chatbot económico del BCRP, puedes consultar sobre cualquier
información respecto a los sectores del pbi
You: dime el pbi de enero 2020
                              Gti_Bot: El PBI PBI tiene variaciones porcentuales anualizadas de 2.98   
You: dime el pbi de marzo 2010
                              Gti_Bot: El PBI PBI tiene variaciones porcentuales anualizadas de 8.13   
You: dime el pbi de marzo 2020
                               Gti_Bot: disculpe, pero esa información todavía no está disponible en el BCRP Data codigo :PN01728AM confidence : 0.9999998807907104 
You: salir


In [49]:
cd ..

/content/drive/My Drive/Colab Notebooks/Chatbot_BCRP


In [0]:
!touch README.md